In [2]:
# Libraries
import json, pandas as pd, pydeck as pdk, ee, ipyfilechooser, ipywidgets, datetime, sys
sys.path.append('./utils/')
import sentinel_satellites

In [3]:
# Initializes the Google Earth Engine APIs
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Features Extraction

The project aim is to create a Machine Learning model capable of detecting the dates when a crop field has been manured, using satellite data. <br>
Before even starting considering models or even doing an exploratory analysis, it is useful to extract all the features (both optical and radar) from sentinel satellites over the specified crop fields.

## Import a JSON file containing crop fields details

In [4]:
# Choose the file (it must be a JSON file)
file_chooser = ipyfilechooser.FileChooser(path='../Datasets/main/', filename='main-crops.json', select_default=True, use_dir_icons=True, filter_pattern='*.json')
display(file_chooser)

FileChooser(path='/home/francesco/Documents/University/master-thesis/Datasets/main', filename='main-crops.json…

In [5]:
# Load JSON data from file
with open(file_chooser.selected) as f:
    data = json.load(f)

# Create DataFrame with properties excluding 'manure_dates' column
fields_df = pd.DataFrame([{k:v for k,v in f['properties'].items() if k!='manure_dates'} for f in data['features']])

# Add column with coordinates for each field
fields_df['polygon_coordinates'] = [[tuple(c) for c in p] for f in data['features'] for p in f['geometry']['coordinates']]

# Create a dataframe that just has the columns crop_field_name and manure_dates
y_df = pd.DataFrame([{k:v for k,v in f['properties'].items() if k!='polygon_coordinates'} for f in data['features']])

In [6]:
# Show the entire dataframe
entire_df = fields_df.merge(y_df, on='crop_field_name')
entire_df

,crop_field_name,polygon_coordinates,manure_dates
0,P-BLD,"[(-4.202723286616649, 43.39683579015289), (-4....",[2022-05-26]
1,P-BLLT1,"[(-4.085622203603083, 43.429605845026266), (-4...",[2022-05-16]
2,P-BLLT2,"[(-4.084840437376829, 43.430826294936246), (-4...",[2022-05-26]
3,P-Cardana,"[(8.658803437240303, 45.85842753378426), (8.65...",[2022-02-24]
4,P-CBRCS1,"[(-4.200826431306206, 43.39067464298489), (-4....",[2022-05-26]
5,P-CBRCS2,"[(-4.204911872695676, 43.3876170244562), (-4.2...",[2022-05-26]
6,P-CLGT,"[(-4.111699726693341, 43.39830644556494), (-4....",[2022-05-16]
7,P-CLMBRS,"[(-4.544769098140127, 43.38040395682432), (-4....",[2022-05-26]
8,P-CMNTR,"[(-4.147208715069137, 43.40038457218137), (-4....",[2022-05-16]
9,P-DR,"[(-4.142486752802821, 43.396858931472195), (-4...",[2022-03-21]


## Show crop fields locations on earth-map

In [7]:
# Define the layer with a tooltip
layer = pdk.Layer(
    'PolygonLayer',
    data=entire_df,
    get_polygon='polygon_coordinates',
    get_fill_color=[255, 255, 0, 100],
    get_line_color=[255, 255, 0, 100],
    stroked=True,
    filled=True,
    lineWidthMinPixels=3,
    pickable=True,
    auto_highlight=True,
)

# Define the initial view state of the map
view_state = pdk.ViewState(
    longitude=fields_df.polygon_coordinates[0][0][0],
    latitude=fields_df.polygon_coordinates[0][0][1],
    zoom=7.8
)

# Create the map with the layers and the initial view state
r = pdk.Deck(layers=layer, initial_view_state=view_state,)

# Show the map
r.show()


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{\n  "initialViewState": {…

It can be noticed that our fields are placed in the Northern part of Spain. Please consider generalization issue.

## Features extraction - using sentinel-satellites PyPI library

The objective is to generate a dataset that contains for each field, for each time the satellites (sentinel 1 and sentinel 2) have passed on the field (in a period, specified by the user), all the phisical indicators that will be further used to build the final model. <br>
This procedure has been designed to be performed in parallel in order to exploit the computational power of the machine (since each field is indipendent with the others).

In [8]:
start_date_widget = ipywidgets.widgets.DatePicker(description='Start date', value=datetime.date(2022, 1, 1), disabled=False)
display(start_date_widget)

end_date_widget = ipywidgets.widgets.DatePicker(description='End date', value=datetime.date(2022, 12, 31), disabled=False)
display(end_date_widget)

DatePicker(value=datetime.date(2022, 1, 1), description='Start date', step=1)

DatePicker(value=datetime.date(2022, 12, 31), description='End date', step=1)

### Sentinel 2 (optical features)

In [15]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using sentinel 2 satellites
fields_s2_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=2, fields_threads=3)
# Add manure dates
fields_s2_features_extracted_df = fields_s2_features_extracted_df.merge(y_df, on=str(y_df.columns[0]))

# Show the dataframe
fields_s2_features_extracted_df

#### Store the obtained dataset

In [ ]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_s2_features_extracted_df.to_csv(filename + '-s2-features-extracted.gz', header=True, index=False, compression='gzip')

### Sentinel 1 (radar features)

In [ ]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using sentinel 1 satellites
fields_s1_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=1, fields_threads=18)
# Add manure dates
fields_s1_features_extracted_df = fields_s1_features_extracted_df.merge(y_df, on=str(y_df.columns[0]))

# Show the dataframe
fields_s1_features_extracted_df

#### Store the obtained dataset

In [ ]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_s1_features_extracted_df.to_csv(filename + '-s1-features-extracted.gz', header=True, index=False, compression='gzip')